In [2]:
import pandas as pd
import sqlalchemy as sa
import urllib
from datetime import datetime

# Data extract and data inspection

In [3]:

df = pd.read_csv('superStore.csv', sep=None, engine='python', encoding='windows-1252')
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Year,Month
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,2016,11
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,2016,11
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,2016,6
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,2015,10
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,2015,10


In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   str    
 2   Order Date     9994 non-null   str    
 3   Ship Date      9994 non-null   str    
 4   Ship Mode      9994 non-null   str    
 5   Customer ID    9994 non-null   str    
 6   Customer Name  9994 non-null   str    
 7   Segment        9994 non-null   str    
 8   Country        9994 non-null   str    
 9   City           9994 non-null   str    
 10  State          9994 non-null   str    
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   str    
 13  Product ID     9994 non-null   str    
 14  Category       9994 non-null   str    
 15  Sub-Category   9994 non-null   str    
 16  Product Name   9994 non-null   str    
 17  Sales          9994 non-null   float64
 18  Quantity       9994

In [5]:
df.isnull().sum()

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
Year             0
Month            0
dtype: int64

# Cleanning ,Transformation and Normailization

In [6]:
# Column names ko standardize karna
df.columns = [c.replace(' ','_').replace('-','_').title() for c in df.columns]

In [7]:
df.columns

Index(['Row_Id', 'Order_Id', 'Order_Date', 'Ship_Date', 'Ship_Mode',
       'Customer_Id', 'Customer_Name', 'Segment', 'Country', 'City', 'State',
       'Postal_Code', 'Region', 'Product_Id', 'Category', 'Sub_Category',
       'Product_Name', 'Sales', 'Quantity', 'Discount', 'Profit', 'Year',
       'Month'],
      dtype='str')

In [8]:
# Date Format Fix karna
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'])

In [9]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row_Id         9994 non-null   int64         
 1   Order_Id       9994 non-null   str           
 2   Order_Date     9994 non-null   datetime64[us]
 3   Ship_Date      9994 non-null   datetime64[us]
 4   Ship_Mode      9994 non-null   str           
 5   Customer_Id    9994 non-null   str           
 6   Customer_Name  9994 non-null   str           
 7   Segment        9994 non-null   str           
 8   Country        9994 non-null   str           
 9   City           9994 non-null   str           
 10  State          9994 non-null   str           
 11  Postal_Code    9994 non-null   int64         
 12  Region         9994 non-null   str           
 13  Product_Id     9994 non-null   str           
 14  Category       9994 non-null   str           
 15  Sub_Category   9994 non-null   s

#Normalizing Data into 5 Tables

In [10]:
#TABLE 1: DIM_LOCATION
dim_location = df[['Country','City','State','Postal_Code']].drop_duplicates().reset_index(drop=True)
dim_location['Location_ID'] = dim_location.index+1
df = pd.merge(df, dim_location, on=['Country','City','State','Postal_Code'], how='left')

In [11]:
dim_location

,Country,City,State,Postal_Code,Location_ID
0,United States,Henderson,Kentucky,42420,1
1,United States,Los Angeles,California,90036,2
2,United States,Fort Lauderdale,Florida,33311,3
3,United States,Los Angeles,California,90032,4
4,United States,Concord,North Carolina,28027,5
...,...,...,...,...,...
627,United States,Springdale,Arkansas,72762,628
628,United States,Lodi,California,95240,629
629,United States,La Porte,Texas,77571,630
630,United States,Mason,Ohio,45040,631


In [12]:
df.head(2)

,Row_Id,Order_Id,Order_Date,Ship_Date,Ship_Mode,Customer_Id,Customer_Name,Segment,Country,City,...,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit,Year,Month,Location_ID
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,2016,11,1
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820,2016,11,1


In [13]:
#TABLE 2: DIM_REGION
dim_region = df[['Region']].drop_duplicates().reset_index(drop=True)
dim_region['Region_Id'] = dim_region.index+1
df = pd.merge(df,dim_region,on='Region',how='left')

In [14]:
dim_region

,Region,Region_Id
0,South,1
1,West,2
2,Central,3
3,East,4


In [15]:
#TABLE 3: DIM_CUSTOMER
dim_Customers = df[['Customer_Id','Customer_Name','Segment']].drop_duplicates(subset=['Customer_Id'],keep='first')


In [16]:
dim_Customers.head()

,Customer_Id,Customer_Name,Segment
0,CG-12520,Claire Gute,Consumer
2,DV-13045,Darrin Van Huff,Corporate
3,SO-20335,Sean O'Donnell,Consumer
5,BH-11710,Brosina Hoffman,Consumer
12,AA-10480,Andrew Allen,Consumer


In [ ]:
#TABLE 4: DIM_PRODUCT
dim_Products = df[['Product_Id','Product_Name','Category','Sub_Category']].drop_duplicates(subset=['Product_Id'],keep='first')
dim_Products.head()

,Product_Id,Product_Name,Category,Sub_Category
0,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels
3,FUR-TA-10000577,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables
4,OFF-ST-10000760,Eldon Fold 'N Roll Cart System,Office Supplies,Storage


In [21]:
#TABLE 5: fact_sales
fact_sales = df[[
    'Order_Id', 
    'Order_Date', 
    'Ship_Date', 
    'Ship_Mode', 
    'Customer_Id',  
    'Product_Id',  
    'Location_ID',  
    'Region_Id',    
    'Sales', 
    'Quantity', 
    'Discount', 
    'Profit'
]]
fact_sales.head(2)

,Order_Id,Order_Date,Ship_Date,Ship_Mode,Customer_Id,Product_Id,Location_ID,Region_Id,Sales,Quantity,Discount,Profit
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,FUR-BO-10001798,1,1,261.96,2,0.0,41.9136
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,FUR-CH-10000454,1,1,731.94,3,0.0,219.5820


In [22]:
print("Normalization Complete!")
print(f"Locations: {len(dim_location)}, Regions: {len(dim_region)}, Customers: {len(dim_Customers)}, Products: {len(dim_Products)}, Orders: {len(fact_sales)}")

Normalization Complete!
Locations: 632, Regions: 4, Customers: 793, Products: 1862, Orders: 9994


# LOAD TO SQL SERVER

In [23]:
#CONFIGURATION
Server_Name = 'mittal-2003\SQLEXPRESS'
Database_Name = 'SuperStore_DB'
params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={Server_Name};"
    f"DATABASE={Database_Name};"
    f"Trusted_Connection=yes;"
)
engine = sa.create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\User\AppData\Local\Temp\ipykernel_17600\3156715043.py:2: SyntaxWarning: invalid escape sequence '\S'
  Server_Name = 'mittal-2003\SQLEXPRESS'


In [24]:
tables_to_load = {
    'Dim_Location': dim_location,
    'Dim_Region' : dim_region,
    'Dim_Customer' : dim_Customers,
    'Dim_Product' : dim_Products,
    'Fact_Sales' : fact_sales
}

for table_name,dataframe in tables_to_load.items():
    try:
        print(f"Loading table: {table_name}...")
        dataframe.to_sql(table_name,con=engine,index=False,if_exists='replace',chunksize=1000)
        print(f"Successfully loaded {table_name}")
    except Exception as e:
        print(f"Error loading {table_name}: {e}")

print("==========================================")
print("ETL PIPELINE COMPLETED SUCCESSFULLY! 🚀")
print("Check your SQL Server Database now.")
print("==========================================")

Loading table: Dim_Location...


c:\Users\User\OneDrive\Desktop\sql project\.venv\Lib\site-packages\pandas\io\sql.py:1649: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())


Successfully loaded Dim_Location
Loading table: Dim_Region...
Successfully loaded Dim_Region
Loading table: Dim_Customer...
Successfully loaded Dim_Customer
Loading table: Dim_Product...
Successfully loaded Dim_Product
Loading table: Fact_Sales...
Successfully loaded Fact_Sales
ETL PIPELINE COMPLETED SUCCESSFULLY! 🚀
Check your SQL Server Database now.
